In [1]:
import pandas as pd
import numpy as np
import names

In [2]:
df = pd.read_csv('datasets/jcpenney/jcpenney.csv')

In [3]:
df.drop(['sku', 'sale_price', 'category', 'product_url', 'total_number_reviews'], inplace=True, axis=1)
df.rename(columns={'uniq_id': 'id', 'name_title': 'name', 'list_price': 'price', 'average_product_rating': 'rating', 'product_image_urls': 'image', 'Reviews': 'reviews', 'category_tree': 'category'}, inplace=True)

In [4]:
# DELETE DUPLICATED ROWS
to_mantain_index = []
to_mantain_name = []
def deleteRows(row):
    if row['name'] not in to_mantain_name:
        to_mantain_index.append(row.name)
        to_mantain_name.append(row['name'])

df.apply(deleteRows, axis=1)
df = df.loc[to_mantain_index]

In [5]:
# RATING
df['rating'] = df['rating'].str.replace(' out of 5','')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [6]:
df['category'] = df['category'].str.split('|')
df['stock'] = np.random.randint(0, 300, df.shape[0])
df['store'] = 'jcpenney'

In [7]:
def parseReviews(row):
    try:
        st = row.reviews[13:-3]
        st = st.split('}, {')

        reviews = []
        i = 1
        for s in st:
            string = '"review_' + str(i) + '"=>'
            review = s.replace(string, '')
            review = review[1:-1]
            review = " ".join(review.split())
            i += 1
            aux = {'author': names.get_full_name(), 'rating': np.round(np.random.uniform(0.1, 5.0), 1), 'date': '2018-2-5', 'title': "", 'body': review}
            reviews.append(aux)
        return reviews
    except:
        return []

df['reviews'] = df.apply(parseReviews, axis=1)

In [ ]:
df.to_csv('datasets/jcpenney/products_jcpenney.csv', encoding='utf-8', index=False)